In [1]:

import torch
import numpy as np
import matplotlib.pyplot as plt
import os
from einops import rearrange
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch import nn
from typing import Any
import glob
import re
from datetime import datetime
import random
from collections import defaultdict
import numpy as np
import onnxruntime

import cv2

random.seed(42)

In [2]:
from models.allcnn2d import AllCNN2D, AllCNN2D_Prod
from drawing.interactive import draw_image

# Global

In [3]:
DEVICE: str = "cuda" if torch.cuda.is_available() else "cpu"

# Paths

In [4]:
file_path: str = os.path.abspath(".")
root_path: str = os.path.join(file_path, os.pardir, os.pardir)
checkpoint_path: str = os.path.join(
    root_path, 
    "checkpoints", 
    "GeckoFull_epoch1980_trainacc0.90605_valacc0.98089_Tloss0.78068_Vloss0.20101_lr6.320194197753456e-11.pkl"
)


# Load Model

In [5]:
alphabet: list[str] = ['(', ')', '+', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'λ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '×', '÷']

model: AllCNN2D_Prod = AllCNN2D_Prod(
    labels_map=alphabet,
    **{
        "conv_features": (1, 16, 32, 32, 32, 32),
        "fully_connected_features": (64, 44),
        "expected_input_size": (64, 64),
        "device": "cpu",
        "conv_dropout": 0.0,
        "verbose": True,
        "name_prefix": "GeckoFinal",
        #"checkpoint_path": checkpoint_path
    }

).eval()

Layer (type:depth-idx)                   Output Shape              Param #
AllCNN2D_Prod                            [1, 44]                   --
├─ModuleList: 1-1                        --                        --
│    └─Sequential: 2-1                   [1, 16, 32, 32]           --
│    │    └─Conv2d: 3-1                  [1, 16, 64, 64]           160
│    │    └─Dropout2d: 3-2               [1, 16, 64, 64]           --
│    │    └─BatchNorm2d: 3-3             [1, 16, 64, 64]           32
│    │    └─LeakyReLU: 3-4               [1, 16, 64, 64]           --
│    │    └─Conv2d: 3-5                  [1, 16, 32, 32]           2,320
│    │    └─Dropout2d: 3-6               [1, 16, 32, 32]           --
│    │    └─BatchNorm2d: 3-7             [1, 16, 32, 32]           32
│    │    └─LeakyReLU: 3-8               [1, 16, 32, 32]           --
│    └─Sequential: 2-2                   [1, 32, 16, 16]           --
│    │    └─Conv2d: 3-9                  [1, 32, 32, 32]           4,640
│    │  

In [6]:
[(i, char, ord(char)) for i, char in enumerate(alphabet)]

[(0, '(', 40),
 (1, ')', 41),
 (2, '+', 43),
 (3, '-', 45),
 (4, '.', 46),
 (5, '0', 48),
 (6, '1', 49),
 (7, '2', 50),
 (8, '3', 51),
 (9, '4', 52),
 (10, '5', 53),
 (11, '6', 54),
 (12, '7', 55),
 (13, '8', 56),
 (14, '9', 57),
 (15, 'λ', 955),
 (16, 'a', 97),
 (17, 'b', 98),
 (18, 'c', 99),
 (19, 'd', 100),
 (20, 'e', 101),
 (21, 'f', 102),
 (22, 'g', 103),
 (23, 'h', 104),
 (24, 'i', 105),
 (25, 'j', 106),
 (26, 'k', 107),
 (27, 'l', 108),
 (28, 'm', 109),
 (29, 'n', 110),
 (30, 'o', 111),
 (31, 'p', 112),
 (32, 'q', 113),
 (33, 'r', 114),
 (34, 's', 115),
 (35, 't', 116),
 (36, 'u', 117),
 (37, 'v', 118),
 (38, 'w', 119),
 (39, 'x', 120),
 (40, 'y', 121),
 (41, 'z', 122),
 (42, '×', 215),
 (43, '÷', 247)]

In [7]:

# Create a dummy input tensor
dummy_input = torch.randn(1, 1, 64, 64)  # Example input

# Define the ONNX file path
onnx_file_path = "model.onnx"

# Export the model
torch.onnx.export(
    model,
    dummy_input,
    onnx_file_path,
    input_names=["input"],  # Name of the input layer
    output_names=["logits", "softmax", "softmax_ordered"],  # Names of the output layers
    dynamic_axes={"input": {0: "batch_size"},  # Variable batch size
                  "logits": {0: "batch_size"},
                  "softmax": {0: "batch_size"},
                  "softmax_ordered": {0: "batch_size"}},
    opset_version=11  # Specify the ONNX opset version
)

print(f"Model saved to {onnx_file_path}")

c:\Users\LeonBass\Documents\visual_studio_code\ocr-model-training\src\cnn\models\allcnn2d.py:568: TracerWarning: Converting a tensor to a Python list might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  softmaxed_char.tolist(),
c:\Users\LeonBass\Documents\visual_studio_code\ocr-model-training\src\cnn\models\allcnn2d.py:580: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  softmax_ordered: Tensor = tensor(


Model saved to model.onnx


# Load Onnx

In [8]:


# Load the ONNX model
model_path = "model.onnx"
session = onnxruntime.InferenceSession(model_path)

input_image = np.random.random(
    (
        1,  # batch: stack as many images as you like here
        1,  # channels: needs to be 1 (grayscale), pixels are 1.0 or 0.0
        64, # height: fixed to 64 pixels for now
        64  # width: fixed to 64 pixels for now
    )
).astype(np.float32)

# Run inference
inputs: list[onnxruntime.NodeArg] = session.get_inputs()
outputs: list[onnxruntime.NodeArg] = session.get_outputs()

input_name: list[str] = inputs[0].name
output_names: list[str] = [out.name for out in outputs]


In [9]:

softmax: np.ndarray
softmax_ordered: np.ndarray
logits: np.ndarray

logits, softmax, softmax_ordered = session.run(
    output_names, 
    {input_name: input_image}
)

# logits.shape is shape (batch, character) for all character labels
# softmax.shape is shape (batch, character) for all character labels
# softmax_ordered is shape (batch, character, [label index, label prob, unicode character value])

# character dim is 44 (there are 44 character labels)
# label index is from 0 to 44 (corresponding to each ordered label index)
# label prob is a softmaxed probability for this label prediction
# unicode character value is the unicode character for this prediction



In [10]:
logits.shape, softmax.shape, softmax_ordered.shape

((1, 44), (1, 44), (1, 44, 3))

In [11]:


top_character_probs: list[list[float]] = softmax_ordered[:, :, 1].tolist()

top_characters: list[list[str]] = [
    [
        chr(int(softmax_ordered[batch_i, i, 2])) 
        for i in range(softmax_ordered.shape[1])
    ] for batch_i in 
    range(softmax_ordered.shape[0])
]


# Hello World

In [12]:
class TextReShitter(nn.Module):
    
    def __init__(self, alphabet: list[str], text: str):
        
        super(TextReShitter, self).__init__()
        
        self.alphabet: list[str] = sorted(alphabet)
        self.text: str = text
        
        self.logits: list[torch.Tensor] = []
        
        for c in self.text:
            c_index: int = self.alphabet.index(c)
            
            char_logits: torch.Tensor = torch.rand((len(alphabet),))
            char_logits[c_index] += 2
            char_logits *= (torch.rand(1)+0.1)

            self.logits.append(char_logits)

        self.logits_tensor: torch.Tensor = torch.stack(self.logits, dim=0)
        
        self.logits_tensor = self.logits_tensor.unsqueeze(0) # batch, chars, logit
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        self.logits_tensor = self.logits_tensor + x.sum() * 0.0  

        softmax = torch.softmax(self.logits_tensor, dim=-1)
        
        # Create indices for the first dimension (char_prob_i)
        char_prob_indices = torch.arange(softmax.size(-1), device=softmax.device).reshape(1, 1, -1, 1)
        
        # Create indices for the third dimension (ord(self.alphabet[char_prob_i]))
        alphabet_indices = torch.tensor([ord(c) for c in self.alphabet], device=softmax.device).reshape(1, 1, -1, 1)
        
        # Expand dimensions to match softmax shape
        char_prob_indices = char_prob_indices.expand(*softmax.shape, 1)
        alphabet_indices = alphabet_indices.expand(*softmax.shape, 1)
        
        # Concatenate along the last dimension
        softmax_ordered = torch.cat([char_prob_indices, softmax.unsqueeze(-1), alphabet_indices], dim=-1)
        
        # Sort along the probability dimension (dim=-2)
        sorting_indices = softmax_ordered[..., 1].argsort(dim=-1, descending=True)
        sorted_tensor = torch.gather(softmax_ordered, -2, sorting_indices.unsqueeze(-1).expand(-1, -1, -1, 3))
        
        x = x * 0.5
        
        return self.logits_tensor, softmax, sorted_tensor
        
        
        

In [13]:
hello_world_model = TextReShitter(alphabet, "hello.world")

logits, softmax, softmax_ord = hello_world_model.forward(torch.zeros(1, 1, 1, 1))

logits.shape, softmax.shape, softmax_ord.shape

(torch.Size([1, 11, 44]), torch.Size([1, 11, 44]), torch.Size([1, 11, 44, 3]))

# ONNX: Save Hello World

In [14]:
# Create a dummy input tensor
dummy_input = torch.randn(1, 1, 1, 1)  # Example input

# Define the ONNX file path
onnx_file_path = "hello_world.onnx"

# Export the model
torch.onnx.export(
    hello_world_model,
    dummy_input,
    onnx_file_path,
    input_names=["input"],  # Name of the input layer
    output_names=["logits", "softmax", "softmax_ordered"],  # Names of the output layers
    dynamic_axes={"input": {0: "batch_size", 2: "height", 3: "width"},  # Variable batch size
                  "logits": {0: "batch_size"},
                  "softmax": {0: "batch_size"},
                  "softmax_ordered": {0: "batch_size"}},
    opset_version=11  # Specify the ONNX opset version
)

print(f"Model saved to {onnx_file_path}")

C:\Users\LeonBass\AppData\Local\Temp\ipykernel_6072\3231084438.py:35: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  alphabet_indices = torch.tensor([ord(c) for c in self.alphabet], device=softmax.device).reshape(1, 1, -1, 1)


Model saved to hello_world.onnx


# Load Hello World Onnx

In [19]:

# Load the ONNX model
model_path = "hello_world_dyn.onnx"
session = onnxruntime.InferenceSession(model_path)

input_image = np.random.random(
    (
        1,  # batch: stack as many images as you like here
        1,  # channels: needs to be 1 (grayscale), pixels are 1.0 or 0.0
        2,  # height: fixed to 1 for now
        1   # width: fixed to 1 for now
    )
).astype(np.float32)

# Run inference
inputs: list[onnxruntime.NodeArg] = session.get_inputs()
outputs: list[onnxruntime.NodeArg] = session.get_outputs()

input_name: str = inputs[0].name
output_names: list[str] = [out.name for out in outputs]
softmax: np.ndarray
softmax_ordered: np.ndarray
logits: np.ndarray

logits, softmax, softmax_ordered = session.run(
    output_names, 
    {input_name: input_image}
)

top3_preds: np.ndarray = softmax_ordered[:, :, :3, 2] # top three predictions for each character (unicode value, convert to string with chr)
top3_pred_probs: np.ndarray = softmax_ordered[:, :, :3, 1] # top three prediction probabilities

print(top3_preds)

print(top3_pred_probs)

for letter_i in range(top3_preds.shape[1]):
    for k in range(3):
        
        pred_percentage: float = top3_pred_probs[0, letter_i, k]*100
        
        print(f"#{k+1} @ {pred_percentage:.4}% - {chr(int(top3_preds[0, letter_i, k]))}")
    print("--")


[[[104. 120. 115.]
  [101. 105. 109.]
  [108.  45. 118.]
  [108.  50.  55.]
  [111. 247.  41.]
  [ 46. 111.  52.]
  [119. 104. 118.]
  [111. 117.  53.]
  [114.  57.  45.]
  [108.  97. 112.]
  [100. 111. 104.]]]
[[[0.13487485 0.03051421 0.03024892]
  [0.03483254 0.02470074 0.0245183 ]
  [0.12618831 0.03325844 0.03256983]
  [0.14055243 0.02988121 0.02889052]
  [0.03572701 0.02537489 0.02510708]
  [0.0649533  0.0259773  0.02597003]
  [0.03041911 0.02436568 0.02433025]
  [0.09235369 0.03029493 0.03025205]
  [0.08532248 0.03095609 0.0284572 ]
  [0.03220282 0.02440088 0.02422533]
  [0.02972033 0.02469328 0.0244364 ]]]
#1 @ 13.49% - h
#2 @ 3.051% - x
#3 @ 3.025% - s
--
#1 @ 3.483% - e
#2 @ 2.47% - i
#3 @ 2.452% - m
--
#1 @ 12.62% - l
#2 @ 3.326% - -
#3 @ 3.257% - v
--
#1 @ 14.06% - l
#2 @ 2.988% - 2
#3 @ 2.889% - 7
--
#1 @ 3.573% - o
#2 @ 2.537% - ÷
#3 @ 2.511% - )
--
#1 @ 6.495% - .
#2 @ 2.598% - o
#3 @ 2.597% - 4
--
#1 @ 3.042% - w
#2 @ 2.437% - h
#3 @ 2.433% - v
--
#1 @ 9.235% - o
#2 @ 3.0